# Plan de Pruebas - API REST con FastAPI

## 📋 Información General

**Base URL:** `http://localhost:8000`  
**Objetivo:** Validar endpoints unitaria e integralmente usando `requests` y `pandas`

---

## 🎯 Endpoints a Evaluar

| Método | Endpoint | Propósito | Evaluado |
|--------|----------|-----------|-----------| 
| GET | `/health` | Verificar disponibilidad del servicio | SI |
| POST | `/matches/testing-text` | Calcular similitud entre dos strings | SI |
| POST | `/matches/compare-by-ids` | Comparar items por ID usando modelo | SI |
| GET | `/tables/{table_name}/colnames` | Obtener nombres de columnas | SI |
| GET | `/tables/{table_name}/header` | Obtener primeros registros de tabla | SI |
| POST | `/tables/add-items` | Insertar/actualizar items (Upsert) | SI |
| POST | `/tables/matches/backup-and-reset` | Respaldar y limpiar tabla matches | SI |

---

## 🧪 Estrategia de Pruebas

### Pruebas Unitarias (Aislamiento)
- Validar respuestas HTTP esperadas (200, 201, 404, 422)
- Verificar estructura de datos (schemas Pydantic)
- Probar casos de error controlados

### Pruebas Integrales (Flujo E2E)
1. **Crear items** → `/tables/add-items`
2. **Comparar por IDs** → `/matches/compare-by-ids`
3. **Verificar registro** → `/tables/matches/header`
4. **Ejecutar backup** → `/tables/matches/backup-and-reset`
5. **Validar persistencia** → Verificar `matches_backup`

---

## ⚠️ Casos de Prueba Críticos

### 🔴 Upsert Logic (Create or Update)
Validar comportamiento con IDs duplicados:
- Primera inserción: `201 Created`
- Actualización con mismo ID: `200 OK` (sin `IntegrityError`)

### 🔴 Backup & Reset Flow
```
Estado Inicial: matches con N registros
↓ POST /backup-and-reset
Estado Final: matches vacía + matches_backup con N registros
```

### 🔴 Validación de Constraints
Verificar integridad de PK compuesta en tabla `items`:
- Columnas requeridas: `id`, `id_item`

---

## 📊 Visualización de Resultados

Se utilizará `pandas.DataFrame` para presentar:
- Respuestas de endpoints tipo lista (colnames, header)
- Comparación antes/después en operaciones de backup
- Resumen de pruebas ejecutadas con estado (✅ PASS / ❌ FAIL)

---

## 🛠️ Preparación del Entorno

**Requisitos:**
```python
pip install requests pandas
```

**Estructura del Notebook:**
1. 📦 Configuración e Imports
2. 🔧 Funciones Helper
3. 🧪 Pruebas Unitarias por Endpoint
4. 🔄 Escenario de Prueba Integral
5. 📈 Reporte de Resultados

---

## ✅ Criterios de Aceptación

- [x] Todos los endpoints responden correctamente
- [x] Lógica Upsert validada sin errores de duplicados
- [x] Flujo de backup preserva datos correctamente
- [x] Casos de error (404, 422) manejados apropiadamente
- [x] Visualización clara de resultados en DataFrames

----

## 🧪 Ejecución de Pruebas

### 📍 Preparacion de entorno de pruebas

In [50]:
import requests
import time

In [51]:

# Configuración
BASE_URL = "http://localhost:8000"

### 📍 Pruebas Unitarias por Endpoint

---

#### 1️⃣ GET `/health`
**Objetivo:** Verificar disponibilidad del servicio

| # | Escenario | Request | Respuesta Esperada | Validación |
|---|-----------|---------|-------------------|------------|
| 1 | Servicio activo | `GET /health` | `200 OK` | `{"status": "healthy"}` |
| 2 | Verificar estructura JSON | `GET /health` | `200 OK` | Contiene campo `status` |
| 3 | Tiempo de respuesta | `GET /health` | `200 OK` | `response_time < 100ms` |

---

In [47]:
# Test 1️⃣: GET /health
print("=" * 60)
print("TEST 1️⃣: GET /health - Verificar disponibilidad del servicio")
print("=" * 60)

TEST 1️⃣: GET /health - Verificar disponibilidad del servicio


In [48]:

# Escenario 1: Servicio activo
print("\n[Test 1.1] Servicio activo")
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
assert response.status_code == 200, "❌ FAIL: Status code no es 200"
expected_response = {
    "status": "ok",
    "message": "Conectividad con la base de datos verificada exitosamente"
}
assert response.json() == expected_response, "❌ FAIL: Respuesta no coincide"
print("✅ PASS: Servicio activo")



[Test 1.1] Servicio activo
Status Code: 200
Response: {'status': 'ok', 'message': 'Conectividad con la base de datos verificada exitosamente'}
✅ PASS: Servicio activo


![image.png](\img\test_01_01.png)

In [49]:

# Escenario 2: Verificar estructura JSON
print("\n[Test 1.2] Verificar estructura JSON")
response = requests.get(f"{BASE_URL}/health")
data = response.json()
print(f"Response: {data}")
assert "status" in data, "❌ FAIL: Campo 'status' no encontrado"
assert "message" in data, "❌ FAIL: Campo 'message' no encontrado"
print("✅ PASS: Estructura JSON válida")



[Test 1.2] Verificar estructura JSON
Response: {'status': 'ok', 'message': 'Conectividad con la base de datos verificada exitosamente'}
✅ PASS: Estructura JSON válida


![image.png](\img\test_01_02.png)

In [52]:

# Escenario 3: Tiempo de respuesta
print("\n[Test 1.3] Tiempo de respuesta")
start_time = time.time()
response = requests.get(f"{BASE_URL}/health")
response_time = (time.time() - start_time) * 1000  # Convertir a ms
print(f"Tiempo de respuesta: {response_time:.2f}ms")
assert response_time < 100, f"❌ FAIL: Tiempo de respuesta {response_time:.2f}ms > 100ms"
print("✅ PASS: Tiempo de respuesta aceptable")


[Test 1.3] Tiempo de respuesta
Tiempo de respuesta: 6.79ms
✅ PASS: Tiempo de respuesta aceptable


![image.png](\img\test_01_03.png)

In [16]:
print("\n" + "=" * 60)
print("✅ TODOS LOS TESTS DE /health PASARON")
print("=" * 60)


✅ TODOS LOS TESTS DE /health PASARON


---
#### 2️⃣ POST `/matches/testing-text`
**Objetivo:** Calcular similitud entre textos

| # | Escenario | Payload | Respuesta Esperada | Validación |
|---|-----------|---------|-------------------|------------|
| 1 | Textos idénticos | `{"text1": "hello", "text2": "hello"}` | `200 OK` | `similarity ≈ 1.0` |
| 2 | Textos diferentes | `{"text1": "python", "text2": "java"}` | `200 OK` | `0.0 < similarity < 1.0` |
| 3 | Payload incompleto | `{"text1": "test"}` | `422 Unprocessable Entity` | Error de validación |

---

In [17]:
# Test 2️⃣: POST /matches/testing-text
print("=" * 60)
print("TEST 2️⃣: POST /matches/testing-text - Calcular similitud entre textos")
print("=" * 60)


TEST 2️⃣: POST /matches/testing-text - Calcular similitud entre textos


In [53]:

# Escenario 1: Textos idénticos
print("\n[Test 2.1] Textos idénticos")
payload = {"text_1": "hello", "text_2": "hello"} 
response = requests.post(f"{BASE_URL}/matches/testing-text", json=payload)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

assert response.status_code in [200, 201], f"❌ FAIL: Status code {response.status_code} no es 200 o 201"
data = response.json()
assert "similarity" in data or "score" in data, "❌ FAIL: Campo 'similarity' o 'score' no encontrado"
similarity = data["score"] if "score" in data else data["similarity"]
print(f"Similarity: {similarity}")
assert similarity >= 0.99, f"❌ FAIL: Similitud {similarity} no es aproximadamente 1.0"
print("✅ PASS: Textos idénticos con similarity ≈ 1.0")



[Test 2.1] Textos idénticos
Status Code: 201
Response: {'id': 962979, 'id_item_1': 'MLA_TEXT_1771458191_1', 'title_item_1': 'hello', 'id_item_2': 'MLA_TEXT_1771458191_2', 'title_item_2': 'hello', 'score': 1.0, 'status': 'positivo'}
Similarity: 1.0
✅ PASS: Textos idénticos con similarity ≈ 1.0


In [19]:

# Escenario 2: Textos diferentes
print("\n[Test 2.2] Textos diferentes")
payload = {"text_1": "python", "text_2": "java"}
response = requests.post(f"{BASE_URL}/matches/testing-text", json=payload)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
assert response.status_code in [200, 201], f"❌ FAIL: Status code {response.status_code} no es 200 o 201"
data = response.json()
assert "similarity" in data or "score" in data, "❌ FAIL: Campo 'similarity' o 'score' no encontrado"
similarity = data["score"] if "score" in data else data["similarity"]
print(f"Similarity: {similarity}")
assert 0.0 <= similarity < 1.0, f"❌ FAIL: Similitud {similarity} fuera del rango esperado"
print("✅ PASS: Textos diferentes con 0.0 < similarity < 1.0")


[Test 2.2] Textos diferentes
Status Code: 201
Response: {'id': 117923, 'id_item_1': 'MLA_TEXT_1771458346_1', 'title_item_1': 'python', 'id_item_2': 'MLA_TEXT_1771458346_2', 'title_item_2': 'java', 'score': 0.0, 'status': 'negativo'}
Similarity: 0.0
✅ PASS: Textos diferentes con 0.0 < similarity < 1.0


In [56]:
# Escenario 3: Payload incompleto
print("\n[Test 2.3] Payload incompleto")
payload = {"text1": "test"}
response = requests.post(f"{BASE_URL}/matches/testing-text", json=payload)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
assert response.status_code == 422, f"❌ FAIL: Status code {response.status_code} no es 422"
data = response.json()
assert "detail" in data, "❌ FAIL: Campo 'detail' no encontrado en error de validación"
print("✅ PASS: Payload incompleto retorna 422 Unprocessable Entity")


[Test 2.3] Payload incompleto
Status Code: 422
Response: {'detail': [{'type': 'missing', 'loc': ['body', 'text_1'], 'msg': 'Field required', 'input': {'text1': 'test'}}, {'type': 'missing', 'loc': ['body', 'text_2'], 'msg': 'Field required', 'input': {'text1': 'test'}}]}
✅ PASS: Payload incompleto retorna 422 Unprocessable Entity


In [54]:
print("\n" + "=" * 60)
print("✅ TODOS LOS TESTS DE /matches/testing-text PASARON")
print("=" * 60)


✅ TODOS LOS TESTS DE /matches/testing-text PASARON


---

#### 4️⃣ GET `/tables/{table_name}/colnames`
**Objetivo:** Obtener nombres de columnas de una tabla

| # | Escenario | Request | Respuesta Esperada | Validación |
|---|-----------|---------|-------------------|------------|
| 1 | Tabla existente (items) | `GET /tables/items/colnames` | `200 OK` | Lista con columnas: `["id", "id_item", ...]` |
| 2 | Tabla existente (matches) | `GET /tables/matches/colnames` | `200 OK` | Lista no vacía de strings |
| 3 | Tabla inexistente | `GET /tables/fake_table/colnames` | `404 Not Found` | Mensaje de error |

---

---
#### 5️⃣ GET `/tables/{table_name}/header`
**Objetivo:** Obtener primeros registros de una tabla

| # | Escenario | Request | Respuesta Esperada | Validación |
|---|-----------|---------|-------------------|------------|
| 1 | Tabla con datos (limit=5) | `GET /tables/items/header?limit=5` | `200 OK` | Retorna máximo 5 registros |
| 2 | Tabla vacía | `GET /tables/empty_table/header` | `200 OK` | Lista vacía `[]` |
| 3 | Parámetro limit inválido | `GET /tables/items/header?limit=-1` | `422 Unprocessable Entity` | Error de validación |

---

---

#### 6️⃣ POST `/tables/add-items`
**Objetivo:** Insertar o actualizar items (Upsert)

| # | Escenario | Payload | Respuesta Esperada | Validación |
|---|-----------|---------|-------------------|------------|
| 1 | **Crear items nuevos** | `{"table_name": "items", "items": [{"id": 100, "id_item": "A100", "name": "Test Item"}]}` | `201 Created` | `{"created": 1, "updated": 0}` |
| 2 | **Actualizar item existente** | `{"table_name": "items", "items": [{"id": 100, "id_item": "A100", "name": "Updated Name"}]}` | `200 OK` | `{"created": 0, "updated": 1}` |
| 3 | **Campos requeridos faltantes** | `{"table_name": "items", "items": [{"id": 101}]}` | `422 Unprocessable Entity` | Error de validación de schema |

---

---
#### 7️⃣ POST `/tables/matches/backup-and-reset`
**Objetivo:** Respaldar tabla matches y vaciarla

| # | Escenario | Precondición | Respuesta Esperada | Validación |
|---|-----------|--------------|-------------------|------------|
| 1 | Tabla con 10 registros | matches tiene 10 filas | `200 OK` | `{"backed_up": 10, "deleted": 10}` |
| 2 | Tabla ya vacía | matches tiene 0 filas | `200 OK` | `{"backed_up": 0, "deleted": 0}` |
| 3 | Verificar backup creado | Después de operación | `200 OK` | `matches_backup` contiene los registros |

---

---
### 🔄 Pruebas Integrales (Flujo End-to-End)


---



---

#### Escenario 1: **Flujo Completo de Matching**

| Paso | Acción | Endpoint | Validación |
|------|--------|----------|------------|
| 1 | Insertar item A | `POST /tables/add-items` | `201 Created` con `id=200` |
| 3 | Comparar item A vs item B | `POST /matches/compare-by-ids` | `200 OK` con score de similitud |
| 4 | Verificar registro en matches | `GET /tables/matches/header` | Contiene comparación (200, 201) |

---

---

#### Escenario 2: **Validación de Idempotencia Upsert**

| Paso | Acción | Payload | Validación |
|------|--------|---------|------------|
| 1 | Crear item inicial | `{"id": 300, "name": "Original"}` | `201 Created` |
| 2 | Actualizar mismo item | `{"id": 300, "name": "Modified"}` | `200 OK` (sin IntegrityError) |
| 3 | Verificar cambio | `GET /tables/items/header` | Campo `name` = "Modified" |

---

---

#### Escenario 3: **Manejo de Errores en Cadena**

| Paso | Acción | Request | Validación |
|------|--------|---------|------------|
| 1 | Intentar comparar ID inexistente | `compare-by-ids` con `id=9999` | `404 Not Found` |
| 2 | Recuperar con inserción | `POST /tables/add-items` con `id=9999` | `201 Created` |
| 3 | Re-intentar comparación | `compare-by-ids` con `id=9999` | `200 OK` exitoso |

---